In [1]:
# AI summer semester 2025
# extra activity as a replacement of Quiz 3

# Sven Ludwig
member1 = 'sludwi2s'

member2 = 'none2s'

In [2]:
from unified_planning.model import Action
from unified_planning.model import Fluent
from unified_planning.model import InstantaneousAction
from unified_planning.model import Object
from unified_planning.model import Problem
from unified_planning.model import Variable
from unified_planning.shortcuts import And
from unified_planning.shortcuts import BoolType
from unified_planning.shortcuts import Equals
from unified_planning.shortcuts import Forall
from unified_planning.shortcuts import GE
from unified_planning.shortcuts import GT
from unified_planning.shortcuts import Implies
from unified_planning.shortcuts import IntType
from unified_planning.shortcuts import LE
from unified_planning.shortcuts import LT
from unified_planning.shortcuts import Minus
from unified_planning.shortcuts import Not
from unified_planning.shortcuts import OneshotPlanner
from unified_planning.shortcuts import Or
from unified_planning.shortcuts import Plus
from unified_planning.shortcuts import UserType
from unified_planning.io import PDDLWriter
from unified_planning.engines import PlanGenerationResultStatus


In [3]:
problem = Problem('development_of_pieces_in_chess')

# define types in global scope

user_types = {}

# types for the chess board using chess jargon

File = UserType('File') # column
user_types['File'] = File
Rank = UserType('Rank') # row
user_types['Rank'] = Rank

# we consider one super type for the chess pieces

Piece = UserType('Piece')
user_types['Piece'] = Piece

# using chess notation as much as possible

K = UserType('K', Piece)  # King
#K = UserType('K')  # King
user_types['K'] = K

Q = UserType('Q', Piece)  # Queen
#Q = UserType('Q')  # Queen
user_types['Q'] = Q

B = UserType('B', Piece)  # Bishop
#B = UserType('B')  # Bishop
user_types['B'] = B

N = UserType('N', Piece)  # Knight
#N = UserType('N')  # Knight
user_types['N'] = N

R = UserType('R', Piece)  # Rook
#R = UserType('R')  # Rook
user_types['R'] = R

P = UserType('P', Piece)  # Pawn
#P = UserType('P')  # Pawn
user_types['P'] = P


def create_board_objects(problem: Problem) -> (Problem, dict[str, Object], dict[str, Object]):
    file_chars = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
    #rank_chars = ['1', '2', '3', '4', '5', '6', '7', '8']
    rank_chars = ['s', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    file_objects = {file_char: Object(file_char, File) for file_char in file_chars}
    rank_objects = {rank_char: Object(rank_char, Rank) for rank_char in rank_chars}
    problem.add_objects(file_objects.values())
    problem.add_objects(rank_objects.values())
    return problem, file_objects, rank_objects


def file_char_to_index(file_char: str) -> int:
    file_char_to_index_dict = {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8}
    return file_char_to_index_dict[file_char]


def rank_char_to_index(rank_char: str) -> int:
    rank_char_to_index_dict = {'s': 1, 't': 2, 'u': 3, 'v': 4, 'w': 5, 'x': 6, 'y': 7, 'z': 8}
    return rank_char_to_index_dict[rank_char]


def file_index(obj: Object) -> int:
    if not obj.type == File:
        raise ValueError(f"Given object must be of type File. obj={obj}")
    idx = file_char_to_index(obj.name)
    if not (isinstance(idx, int) and 1 <= idx <= 8):
        raise ValueError(f"Error obtaining file index. obj={obj} idx={idx}")
    return idx


def rank_index(obj: Object) -> int:
    if not obj.type == Rank:
        raise ValueError(f"Given object must be of type Rank. obj={obj}")
    idx = rank_char_to_index(obj.name)
    if not (isinstance(idx, int) and 1 <= idx <= 8):
        raise ValueError(f"Error obtaining rank index. obj={obj} idx={idx}")
    return idx


def define_board_fluents(problem: Problem) -> (Problem, Fluent, Fluent):
    # these fluents assign indices to board objects
    file_idx = Fluent('file_idx', IntType(1,8), f=File)
    rank_idx = Fluent('rank_idx', IntType(1,8), r=Rank)
    problem.add_fluent(file_idx)
    problem.add_fluent(rank_idx)
    return problem, file_idx, rank_idx


def initialize_board_fluents(problem: Problem, file_objects: dict[str, Object], rank_objects: dict[str, Object]) -> Problem:
    file_idx = problem.fluent('file_idx')
    rank_idx = problem.fluent('rank_idx')
    # create one file index fluent per file
    for f_obj in file_objects.values():
        problem.set_initial_value(file_idx(f_obj), file_char_to_index(f_obj.name))
    # create one rank index fluent per rank
    for r_obj in rank_objects.values():
        problem.set_initial_value(rank_idx(r_obj), rank_char_to_index(r_obj.name))
    return problem


def define_occupied_fluent(problem: Problem) -> Problem:
    # occupied is redundant with at
    # if we consider exists p at(p, f, r)
    # but many if not all engines do not support quantifiers in preconditions
    v_f_occ = Variable('v_f_occ', File)
    v_r_occ = Variable('v_r_occ', Rank)
    occupied = Fluent('occupied', BoolType(), [v_f_occ, v_r_occ])
    problem.add_fluent(occupied)
    return problem


def initialize_occupied_fluents(problem: Problem,
                                file_objects: dict[str, Object],
                                rank_objects: dict[str, Object]) -> Problem:
    occupied = problem.fluent('occupied')
    for f_obj in file_objects.values():
        for r_obj in rank_objects.values():
            problem.set_initial_value(occupied(f_obj, r_obj), False)
    return problem


def define_free_fluent(problem: Problem) -> Problem:
    # free is redundant with occupied
    # we introduced it trying to get around all restrictions of the pyperplan engine
    v_f_free = Variable('v_f_free', File)
    v_r_free = Variable('v_r_free', Rank)
    free = Fluent('free', BoolType(), [v_f_free, v_r_free])
    problem.add_fluent(free)
    return problem


def initialize_free_fluents(problem: Problem,
                            file_objects: dict[str, Object],
                            rank_objects: dict[str, Object]) -> Problem:
    free = problem.fluent('free')
    for f_obj in file_objects.values():
        for r_obj in rank_objects.values():
            problem.set_initial_value(free(f_obj, r_obj), True)
    return problem


def define_at_fluent(problem: Problem):
    v_p_at = Variable('v_p_at', Piece)
    v_f_at = Variable('v_f_at', File)
    v_r_at = Variable('v_r_at', Rank)
    at = Fluent('at', BoolType(), [v_p_at, v_f_at, v_r_at])
    problem.add_fluent(at)
    return problem


# we introduced this type-specific at fluent
# trying to circumvent restrictions of certain planning engines
#def define_pawn_at_fluent(problem: Problem):
#    v_P_at = Variable('v_P_at', P)
#    v_f_at = Variable('v_f_at', File)
#    v_r_at = Variable('v_r_at', Rank)
#    pawn_at = Fluent('pawn_at', BoolType(), [v_P_at, v_f_at, v_r_at])
#    problem.add_fluent(pawn_at)
#    return problem


def define_moved_fluent(problem: Problem):
    v_p_moved = Variable('v_p_moved', Piece)
    moved = Fluent('moved', BoolType(), [v_p_moved])
    problem.add_fluent(moved)
    return problem


def define_action_move_pawn_up_by_1(problem: Problem) -> (Problem, Action):
    v_q = Variable('v_q', Piece)
    at = problem.fluent('at')
    occupied = problem.fluent('occupied')
    free = problem.fluent('free')
    moved = problem.fluent('moved')
    # move pawn ahead by 1 square
    move_pawn_up_by_1 = InstantaneousAction('move_pawn_up_by_1', piece=Piece, from_file=File, from_rank=Rank, to_file=File, to_rank=Rank)
    p = move_pawn_up_by_1.parameter('piece')
    ff = move_pawn_up_by_1.parameter('from_file')
    fr = move_pawn_up_by_1.parameter('from_rank')
    tf = move_pawn_up_by_1.parameter('to_file')
    tr = move_pawn_up_by_1.parameter('to_rank')

    # pawn is at the from square
    move_pawn_up_by_1.add_precondition(at(p, ff, fr))

    # nothing is at the to square
    # quite a bit of redundancy since different planners have different capabilities
    # e.g. pyperplan does not support quantifiers in preconditions
    #move_pawn_up_by_1.add_precondition(
    #    Forall(Not(at(v_q, tf, tr)), v_q)
    #)
    move_pawn_up_by_1.add_precondition(Not(occupied(tf, tr)))
    move_pawn_up_by_1.add_precondition(free(tf, tr))

    # pawn is not standing unpromoted at rank 8
    move_pawn_up_by_1.add_precondition(LT(rank_idx(fr), 8))

    # outer numeric constraints regarding standard pawn movement up by 1
    move_pawn_up_by_1.add_precondition(GE(file_idx(ff), 2))
    move_pawn_up_by_1.add_precondition(LE(file_idx(ff), 7))
    move_pawn_up_by_1.add_precondition(GE(file_idx(tf), 3))
    move_pawn_up_by_1.add_precondition(LE(file_idx(tf), 8))

    # within same file as pawn is moving vertically
    move_pawn_up_by_1.add_precondition(Equals(ff, tf))
    # moving up by 1 square
    move_pawn_up_by_1.add_precondition(
        Equals(Plus(rank_idx(fr), 1), rank_idx(tr))
    )

    # effects
    # pawn is no longer at from square
    move_pawn_up_by_1.add_effect(at(p, ff, fr), False)
    move_pawn_up_by_1.add_effect(occupied(ff, fr), False)
    move_pawn_up_by_1.add_effect(free(ff, fr), True)
    # pawn is now at to square
    move_pawn_up_by_1.add_effect(at(p, tf, tr), True)
    move_pawn_up_by_1.add_effect(occupied(tf, tr), True)
    move_pawn_up_by_1.add_effect(free(tf, tr), False)
    # pawn has moved at least once
    move_pawn_up_by_1.add_effect(moved(p), True)

    problem.add_action(move_pawn_up_by_1)
    return problem, move_pawn_up_by_1


def populate_pawns(problem: Problem, file_objects: dict[str, Object], rank_objects: dict[str, Object]) -> (Problem, dict[str, Object]):
    # note: black pieces are out of scope for now
    # create one pawn objects per file
    #pawn_objects = {f'P{file_char}': Object(f'P{file_char}', P) for file_char in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']}
    # instantiate pawns not with their concrete type
    # but as Piece since some planners do not support hierarchical typing
    pawn_objects = {f'P{file_char}': Object(f'P{file_char}', Piece) for file_char in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']}
    problem.add_objects(pawn_objects.values())
    # adjust respective fluents
    at = problem.fluent('at')
    occupied = problem.fluent('occupied')
    #pawn_at = problem.fluent('pawn_at')
    moved = problem.fluent('moved')
    for pawn_str, pawn in pawn_objects.items():
        file_char = pawn_str[1]
        problem.set_initial_value(at(pawn, file_objects[file_char], rank_objects['t']), True)
        problem.set_initial_value(occupied(file_objects[file_char], rank_objects['t']), True)
        problem.set_initial_value(moved(pawn), False)
    return problem, pawn_objects


def define_action_move_rook_horizontally(problem: Problem) -> (Problem, Action):
    at = problem.fluent('at')
    occupied = problem.fluent('occupied')
    free = problem.fluent('free')
    moved = problem.fluent('moved')
    file_idx = problem.fluent('file_idx')
    rank_idx = problem.fluent('rank_idx')

    move_rook_horizontally = InstantaneousAction(
        'move_rook_horizontally',
        piece=Piece,
        from_file=File, from_rank=Rank,
        to_file=File, to_rank=Rank
    )

    p = move_rook_horizontally.parameter('piece')
    ff = move_rook_horizontally.parameter('from_file')
    fr = move_rook_horizontally.parameter('from_rank')
    tf = move_rook_horizontally.parameter('to_file')
    tr = move_rook_horizontally.parameter('to_rank')

    # within same rank since rook is moving horizontally
    move_rook_horizontally.add_precondition(Equals(fr, tr))
    # actually moving i.e. not remaining in the same square
    move_rook_horizontally.add_precondition(Not(Equals(ff, tf)))
    # rook is at the from square
    move_rook_horizontally.add_precondition(at(p, ff, fr))
    # nothing is at the to square
    move_rook_horizontally.add_precondition(Not(occupied(tf, tr)))
    move_rook_horizontally.add_precondition(free(tf, tr))

    # constraints based on the board dimensions
    # redundant with IntType(1,8)
    move_rook_horizontally.add_precondition(GE(file_idx(ff), 1))
    move_rook_horizontally.add_precondition(LE(file_idx(ff), 8))
    move_rook_horizontally.add_precondition(GE(file_idx(tf), 1))
    move_rook_horizontally.add_precondition(LE(file_idx(tf), 8))

    # regarding the check if the horizontal path is clear
    v_f_clear = Variable('v_f_clear', File)
    v_r_clear = Variable('v_r_clear', Rank)
    square_to_check_if_clear_between_horizontally = And(
        Equals(fr, tr),# within same rank since rook is moving horizontally
        Equals(v_r_clear, fr), # within same rank since rook is moving horizontally
        Or(
            And(
                # moving east
                LT(file_idx(ff), file_idx(tf)),
                # any square in between we are checking if clear
                # must be east of the from file
                GT(file_idx(v_f_clear), file_idx(ff)),
                # any square in between we are checking if clear
                # must be west of the to file
                # the to file itself is checked elsewhere
                LT(file_idx(v_f_clear), file_idx(tf))
            ),
            And(
                # moving west
                GT(file_idx(ff), file_idx(tf)),
                # any square in between we are checking if clear
                # must be west of the from file
                LT(file_idx(v_f_clear), file_idx(ff)),
                # any square in between we are checking if clear
                # must be east of the to file
                # the to file itself is checked elsewhere
                GT(file_idx(v_f_clear), file_idx(tf))
            )
        )
    )

    # the actual check if the horizontal path is clear
    move_rook_horizontally.add_precondition(
        Forall(
            Implies(square_to_check_if_clear_between_horizontally, free(v_f_clear, v_r_clear)),
            v_f_clear,
            v_r_clear
        )
    )

    # effects
    # rook is no longer at from square
    move_rook_horizontally.add_effect(at(p, ff, fr), False)
    move_rook_horizontally.add_effect(occupied(ff, fr), False)
    move_rook_horizontally.add_effect(free(ff, fr), True)
    # rook is now at to square
    move_rook_horizontally.add_effect(at(p, tf, tr), True)
    move_rook_horizontally.add_effect(occupied(tf, tr), True)
    move_rook_horizontally.add_effect(free(tf, tr), False)
    # rook has moved at least once
    move_rook_horizontally.add_effect(moved(p), True)

    problem.add_action(move_rook_horizontally)
    return problem, move_rook_horizontally


def define_action_move_rook_vertically(problem: Problem) -> (Problem, Action):
    at = problem.fluent('at')
    occupied = problem.fluent('occupied')
    free = problem.fluent('free')
    moved = problem.fluent('moved')
    file_idx = problem.fluent('file_idx')
    rank_idx = problem.fluent('rank_idx')

    move_rook_vert = InstantaneousAction(
        'move_rook_vertically',
        piece=Piece,
        from_file=File, from_rank=Rank,
        to_file=File, to_rank=Rank
    )

    p = move_rook_vert.parameter('piece')
    ff = move_rook_vert.parameter('from_file')
    fr = move_rook_vert.parameter('from_rank')
    tf = move_rook_vert.parameter('to_file')
    tr = move_rook_vert.parameter('to_rank')

    # within same file since rook is moving vertically
    move_rook_vert.add_precondition(Equals(ff, tf))
    # actually moving i.e. not remaining in the same square
    move_rook_vert.add_precondition(Not(Equals(fr, tr)))
    # rook is at the from square
    move_rook_vert.add_precondition(at(p, ff, fr))
    # nothing is at the to square
    move_rook_vert.add_precondition(Not(occupied(tf, tr)))
    move_rook_vert.add_precondition(free(tf, tr))

    # constraints based on the board dimensions
    # redundant with IntType(1,8)
    move_rook_vert.add_precondition(GE(rank_idx(fr), 1))
    move_rook_vert.add_precondition(LE(rank_idx(fr), 8))
    move_rook_vert.add_precondition(GE(rank_idx(tr), 1))
    move_rook_vert.add_precondition(LE(rank_idx(tr), 8))

    # regarding the check if the vertical path is clear
    v_f_clear = Variable('v_f_clear', File)
    v_r_clear = Variable('v_r_clear', Rank)
    square_to_check_if_clear_between_vertically = And(
        Equals(ff, tf), # within same file since rook is moving vertically
        Equals(v_f_clear, ff), # within same file since rook is moving vertically
        Or(
            And(
                # moving up
                LT(rank_idx(fr), rank_idx(tr)),
                # any square in between we are checking if clear
                # must be above the from rank
                GT(rank_idx(v_r_clear), rank_idx(fr)),
                # any square in between we are checking if clear
                # must be below the to rank
                # the to rank itself is checked elsewhere
                LT(rank_idx(v_r_clear), rank_idx(tr))
            ),
            And(
                # moving down
                GT(rank_idx(fr), rank_idx(tr)),
                # any square in between we are checking if clear
                # must be below the from rank
                LT(rank_idx(v_r_clear), rank_idx(fr)),
                # any square in between we are checking if clear
                # must be above the to rank
                # the to rank itself is checked elsewhere
                GT(rank_idx(v_r_clear), rank_idx(tr))
            )
        )
    )

    # the actual check if the vertical path is clear
    move_rook_vert.add_precondition(
        Forall(
            Implies(square_to_check_if_clear_between_vertically, free(v_f_clear, v_r_clear)),
            v_f_clear,
            v_r_clear
        )
    )

    # effects
    # rook is no longer at from square
    move_rook_vert.add_effect(at(p, ff, fr), False)
    move_rook_vert.add_effect(occupied(ff, fr), False)
    move_rook_vert.add_effect(free(ff, fr), True)
    # rook is now at to square
    move_rook_vert.add_effect(at(p, tf, tr), True)
    move_rook_vert.add_effect(occupied(tf, tr), True)
    move_rook_vert.add_effect(free(tf, tr), False)
    # rook has moved at least once
    move_rook_vert.add_effect(moved(p), True)

    problem.add_action(move_rook_vert)
    return problem, move_rook_vert


def populate_rooks(problem: Problem) -> (Problem, dict[str, Object]):
    rook_objects = {}

    # fluents
    at = problem.fluent('at')
    occupied = problem.fluent('occupied')
    free = problem.fluent('free')
    moved = problem.fluent('moved')

    # create rook objects
    # instantiate pawns not with their concrete type
    # but as Piece since some planners do not support hierarchical typing
    rook_a1 = Object('R1', Piece)
    rook_objects['R1'] = rook_a1
    problem.add_object(rook_a1)
    rook_h1 = Object('R2', Piece)
    rook_objects['R2'] = rook_h1
    problem.add_object(rook_h1)

    # get required file and rank objects
    file_a = problem.object('a')
    file_h = problem.object('h')
    rank_1 = problem.object('s')

    # set initial state of rook at a1
    problem.set_initial_value(at(rook_a1, file_a, rank_1), True)
    problem.set_initial_value(occupied(file_a, rank_1), True)
    problem.set_initial_value(free(file_a, rank_1), False)
    problem.set_initial_value(moved(rook_a1), False)

    # set initial state of rook at h1
    problem.set_initial_value(at(rook_h1, file_h, rank_1), True)
    problem.set_initial_value(occupied(file_h, rank_1), True)
    problem.set_initial_value(free(file_h, rank_1), False)
    problem.set_initial_value(moved(rook_h1), False)

    return problem, rook_objects


def define_action_move_knight(problem: Problem) -> (Problem, Action):
    at = problem.fluent('at')
    occupied = problem.fluent('occupied')
    free = problem.fluent('free')
    moved = problem.fluent('moved')
    file_idx = problem.fluent('file_idx')
    rank_idx = problem.fluent('rank_idx')

    move_knight = InstantaneousAction(
        'move_knight',
        piece=Piece,
        from_file=File, from_rank=Rank,
        to_file=File,   to_rank=Rank
    )
    p  = move_knight.parameter('piece')
    ff = move_knight.parameter('from_file')
    fr = move_knight.parameter('from_rank')
    tf = move_knight.parameter('to_file')
    tr = move_knight.parameter('to_rank')

    # knight is at the from square
    move_knight.add_precondition(at(p, ff, fr))
    # nothing is at the to square
    move_knight.add_precondition(Not(occupied(tf, tr)))
    move_knight.add_precondition(free(tf, tr))

    # numerical boundaries based on board dimensions
    # redundant with IntType(1, 8)
    move_knight.add_precondition(GE(file_idx(ff), 1))
    move_knight.add_precondition(LE(file_idx(ff), 8))
    move_knight.add_precondition(GE(rank_idx(fr), 1))
    move_knight.add_precondition(LE(rank_idx(fr), 8))
    move_knight.add_precondition(GE(file_idx(tf), 1))
    move_knight.add_precondition(LE(file_idx(tf), 8))
    move_knight.add_precondition(GE(rank_idx(tr), 1))
    move_knight.add_precondition(LE(rank_idx(tr), 8))

    # actually moving i.e. not remaining at the same square
    move_knight.add_precondition(
        Or(Not(Equals(ff, tf)), Not(Equals(fr, tr)))
    )

    # L-shape pattern generates 8 different knight moves
    # distance in number of files
    df = Minus(file_idx(tf), file_idx(ff))
    # distance in number of ranks
    dr = Minus(rank_idx(tr), rank_idx(fr))
    # it must be one of the 8 combinations of distances
    move_knight.add_precondition(
        Or(
            And(Equals(df, 2), Equals(dr, 1)),   # 2 right 1 up
            And(Equals(df, 2), Equals(dr, -1)),  # 2 right 1 down
            And(Equals(df, -2), Equals(dr, 1)),  # 2 left 1 up
            And(Equals(df, -2), Equals(dr, -1)), # 2 left 1 down
            And(Equals(df, 1), Equals(dr, 2)),   # 1 right 2 up
            And(Equals(df, 1), Equals(dr, -2)),  # 1 right 2 down
            And(Equals(df, -1), Equals(dr, 2)),  # 1 left 2 up
            And(Equals(df, -1), Equals(dr, -2))  # 1 left 2 down
        )
    )

    # effects
    # knight is no longer at the from square
    move_knight.add_effect(at(p, ff, fr), False)
    move_knight.add_effect(occupied(ff, fr), False)
    move_knight.add_effect(free(ff, fr), True)
    # knight is now at the to square
    move_knight.add_effect(at(p, tf, tr), True)
    move_knight.add_effect(occupied(tf, tr), True)
    move_knight.add_effect(free(tf, tr), False)
    # knight has moved at least once
    move_knight.add_effect(moved(p), True)

    problem.add_action(move_knight)
    return problem, move_knight


def populate_knights(problem: Problem) -> (Problem, dict[str, Object]):
   knight_objects = {}

   # fluents
   at = problem.fluent('at')
   occupied = problem.fluent('occupied')
   free = problem.fluent('free')
   moved = problem.fluent('moved')

   # create knight objects
   # instantiate knights not with their concrete type
   # but as Piece since some planners do not support hierarchical typing
   knight_b1 = Object('N1', Piece)
   knight_objects['N1'] = knight_b1
   problem.add_object(knight_b1)
   knight_g1 = Object('N2', Piece)
   knight_objects['N2'] = knight_g1
   problem.add_object(knight_g1)

   # get required file and rank objects
   file_b = problem.object('b')
   file_g = problem.object('g')
   rank_1 = problem.object('s')

   # set initial state of knight at b1
   problem.set_initial_value(at(knight_b1, file_b, rank_1), True)
   problem.set_initial_value(occupied(file_b, rank_1), True)
   problem.set_initial_value(free(file_b, rank_1), False)
   problem.set_initial_value(moved(knight_b1), False)

   # set initial state of knight at g1
   problem.set_initial_value(at(knight_g1, file_g, rank_1), True)
   problem.set_initial_value(occupied(file_g, rank_1), True)
   problem.set_initial_value(free(file_g, rank_1), False)
   problem.set_initial_value(moved(knight_g1), False)

   return problem, knight_objects


def define_action_move_bishop(problem: Problem) -> (Problem, Action):
   at = problem.fluent('at')
   occupied = problem.fluent('occupied')
   free = problem.fluent('free')
   moved = problem.fluent('moved')
   file_idx = problem.fluent('file_idx')
   rank_idx = problem.fluent('rank_idx')

   move_bishop = InstantaneousAction(
       'move_bishop',
       piece=Piece,
       from_file=File, from_rank=Rank,
       to_file=File,   to_rank=Rank
   )
   p  = move_bishop.parameter('piece')
   ff = move_bishop.parameter('from_file')
   fr = move_bishop.parameter('from_rank')
   tf = move_bishop.parameter('to_file')
   tr = move_bishop.parameter('to_rank')

   # bishop is at the from square
   move_bishop.add_precondition(at(p, ff, fr))
   # nothing is at the to square
   move_bishop.add_precondition(Not(occupied(tf, tr)))
   move_bishop.add_precondition(free(tf, tr))

    # numerical boundaries based on board dimensions
    # redundant with IntType(1, 8)
   move_bishop.add_precondition(GE(file_idx(ff), 1))
   move_bishop.add_precondition(LE(file_idx(ff), 8))
   move_bishop.add_precondition(GE(rank_idx(fr), 1))
   move_bishop.add_precondition(LE(rank_idx(fr), 8))
   move_bishop.add_precondition(GE(file_idx(tf), 1))
   move_bishop.add_precondition(LE(file_idx(tf), 8))
   move_bishop.add_precondition(GE(rank_idx(tr), 1))
   move_bishop.add_precondition(LE(rank_idx(tr), 8))

    # actually moving i.e. not remaining at the same square
   move_bishop.add_precondition(
       Or(Not(Equals(ff, tf)), Not(Equals(fr, tr)))
   )

   # a bishop moves diagonally
   # distance in files
   df = Minus(file_idx(tf), file_idx(ff))
   # distance in ranks
   dr = Minus(rank_idx(tr), rank_idx(fr))
   # so the absolute distance in files must equal the absolute distance in ranks
   move_bishop.add_precondition(
       And(
           # non-zero distance in files
           Not(Equals(df, 0)),
           # non-zero distance in ranks
           Not(Equals(dr, 0)),
           Or(
               # for northeast diagonal and southwest diagonal
               # difference in files equals difference in ranks
               Equals(df, dr),
               # for northwest diagonal and southeast diagonal
               # difference in files equals minus the difference in ranks
               Equals(df, Minus(0, dr))
           )
       )
   )

   #  all squares between source and destination must be free
   v_f_clear_bishop = Variable('v_f_clear_bishop', File)
   v_r_clear_bishop = Variable('v_r_clear_bishop', Rank)
   square_to_check_if_clear_between_diagonally = And(
       Or(
           # northeast diagonal
           # both file and rank increase together
           And(
               # moving east file increases
               LT(file_idx(ff), file_idx(tf)),
               # moving up rank also increases
               LT(rank_idx(fr), rank_idx(tr)),
               # any square we check in between
               GT(file_idx(v_f_clear_bishop), file_idx(ff)),
               LT(file_idx(v_f_clear_bishop), file_idx(tf)),
               Equals(Minus(file_idx(v_f_clear_bishop), file_idx(ff)),
                      Minus(rank_idx(v_r_clear_bishop), rank_idx(fr)))
           ),
           # southwest diagonal
           # both file and rank decrease together
           And(
               # moving west file decreases
               GT(file_idx(ff), file_idx(tf)),
               # moving down rank also decreases
               GT(rank_idx(fr), rank_idx(tr)),
               # any square we check in between
               LT(file_idx(v_f_clear_bishop), file_idx(ff)),
               GT(file_idx(v_f_clear_bishop), file_idx(tf)),
               Equals(Minus(file_idx(v_f_clear_bishop), file_idx(ff)),
                      Minus(rank_idx(v_r_clear_bishop), rank_idx(fr)))
           ),
           # northwest diagonal
           # file decreases and rank increases
           And(
               # moving west file decreases
               GT(file_idx(ff), file_idx(tf)),
               # moving up rank increases
               LT(rank_idx(fr), rank_idx(tr)),
               # any square we check in between
               LT(file_idx(v_f_clear_bishop), file_idx(ff)),
               GT(file_idx(v_f_clear_bishop), file_idx(tf)),
               Equals(Minus(file_idx(ff), file_idx(v_f_clear_bishop)),
                      Minus(rank_idx(v_r_clear_bishop), rank_idx(fr)))
           ),
           # southeast diagonal
           # file increases and rank decreases
           And(
               # moving east file increases
               LT(file_idx(ff), file_idx(tf)),
               # and moving down rank decreases
               GT(rank_idx(fr), rank_idx(tr)),
               # any square we check in between
               GT(file_idx(v_f_clear_bishop), file_idx(ff)),
               LT(file_idx(v_f_clear_bishop), file_idx(tf)),
               Equals(Minus(file_idx(v_f_clear_bishop), file_idx(ff)),
                      Minus(rank_idx(fr), rank_idx(v_r_clear_bishop)))
           )
       )
   )

   # the actual check if the diagonal path is clear
   move_bishop.add_precondition(
       Forall(
           Implies(square_to_check_if_clear_between_diagonally, free(v_f_clear_bishop, v_r_clear_bishop)),
           v_f_clear_bishop,
           v_r_clear_bishop
       )
   )

   # effects
   # bishop is no longer at from square
   move_bishop.add_effect(at(p, ff, fr), False)
   move_bishop.add_effect(occupied(ff, fr), False)
   move_bishop.add_effect(free(ff, fr), True)
   # bishop is now at to square
   move_bishop.add_effect(at(p, tf, tr), True)
   move_bishop.add_effect(occupied(tf, tr), True)
   move_bishop.add_effect(free(tf, tr), False)
   # bishop has moved at least once
   move_bishop.add_effect(moved(p), True)

   problem.add_action(move_bishop)
   return problem, move_bishop


def populate_bishops(problem: Problem) -> (Problem, dict[str, Object]):
    at = problem.fluent('at')
    occupied = problem.fluent('occupied')
    free = problem.fluent('free')
    moved = problem.fluent('moved')

    file_c = problem.object('c')  # column c
    file_f = problem.object('f')  # column f
    rank_1 = problem.object('s')  # row 1

    bishop_c1 = Object('B1', Piece)
    bishop_f1 = Object('B2', Piece)
    problem.add_objects([bishop_c1, bishop_f1])
    bishop_objects = {'B1': bishop_c1, 'B2': bishop_f1}

    problem.set_initial_value(at(bishop_c1, file_c, rank_1), True)
    problem.set_initial_value(occupied(file_c, rank_1), True)
    problem.set_initial_value(free(file_c, rank_1), False)
    problem.set_initial_value(moved(bishop_c1), False)

    problem.set_initial_value(at(bishop_f1, file_f, rank_1), True)
    problem.set_initial_value(occupied(file_f, rank_1), True)
    problem.set_initial_value(free(file_f, rank_1), False)
    problem.set_initial_value(moved(bishop_f1), False)

    return problem, bishop_objects


# all types have been defined above in global scope
# here we call above functions to define fluents and create objects

problem, file_objects, rank_objects = create_board_objects(problem)
problem, file_idx, rank_idx = define_board_fluents(problem)
problem = initialize_board_fluents(problem, file_objects, rank_objects)
problem = define_occupied_fluent(problem)
problem = initialize_occupied_fluents(problem, file_objects, rank_objects)
problem = define_free_fluent(problem)
problem = initialize_free_fluents(problem, file_objects, rank_objects)
problem = define_at_fluent(problem)
#problem = define_pawn_at_fluent(problem)
problem = define_moved_fluent(problem)
# pawns
problem, move_pawn = define_action_move_pawn_up_by_1(problem)
problem, pawn_objects = populate_pawns(problem, file_objects, rank_objects)
# rooks
problem, move_rook = define_action_move_rook_horizontally(problem)
problem, move_rook = define_action_move_rook_vertically(problem)
problem, rook_objects = populate_rooks(problem)
# knights
problem, move_rook = define_action_move_knight(problem)
problem, knight_objects = populate_knights(problem)
# bishops
problem, move_rook = define_action_move_bishop(problem)
problem, bishop_objects = populate_bishops(problem)



/opt/miniconda3/envs/ai311/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.27.2 is exactly one major version older than the runtime version 6.31.1 at unified_planning.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(


In [4]:
# goals

def add_goal_pawn_e_to_rank_6(problem: Problem,
                              pawn_objects: dict[str, Object],
                              file_objects: dict[str, Object],
                              rank_objects: dict[str, Object]) -> Problem:
    # this is a very easy goal for testing purposes
    # the (white) pawn in the e file is to reach rank 6
    at = problem.fluent('at')
    problem.add_goal(at(pawn_objects['Pe'], file_objects['e'], rank_objects['x']))
    return problem

def add_goal_both_knights_have_moved(problem: Problem,
                                     knight_objects: dict[str, Object]) -> Problem:
    # this is a very easy goal for testing purposes
    # both (white) knights must have moved at least once
   moved = problem.fluent('moved')
   problem.add_goal(And(
       moved(knight_objects['N1']),
       moved(knight_objects['N2'])
   ))
   return problem

problem = add_goal_pawn_e_to_rank_6(problem, pawn_objects, file_objects, rank_objects)
#problem = add_goal_both_knights_have_moved(problem, knight_objects)


In [5]:
print(f"problem.kind={problem.kind}")

problem.kind=PROBLEM_CLASS: ['ACTION_BASED']
PROBLEM_TYPE: ['SIMPLE_NUMERIC_PLANNING']
NUMBERS: ['BOUNDED_TYPES']
CONDITIONS_KIND: ['EQUALITIES', 'DISJUNCTIVE_CONDITIONS', 'NEGATIVE_CONDITIONS', 'UNIVERSAL_CONDITIONS']
TYPING: ['FLAT_TYPING']
FLUENTS_TYPE: ['INT_FLUENTS']
INITIAL_STATE: ['UNDEFINED_INITIAL_SYMBOLIC']


In [6]:
writer = PDDLWriter(problem)
print(writer.get_domain())

(define (domain development_of_pieces_in_chess-domain)
 (:requirements :strips :typing :negative-preconditions :disjunctive-preconditions :equality :numeric-fluents :universal-preconditions)
 (:types file rank piece)
 (:predicates (occupied ?v_f_occ - file ?v_r_occ - rank) (free ?v_f_free - file ?v_r_free - rank) (at_ ?v_p_at - piece ?v_f_at - file ?v_r_at - rank) (moved ?v_p_moved - piece))
 (:functions (file_idx ?f - file) (rank_idx ?r - rank))
 (:action move_pawn_up_by_1
  :parameters ( ?piece - piece ?from_file - file ?from_rank - rank ?to_file - file ?to_rank - rank)
  :precondition (and (at_ ?piece ?from_file ?from_rank) (not (occupied ?to_file ?to_rank)) (free ?to_file ?to_rank) (< (rank_idx ?from_rank) 8) (<= 2 (file_idx ?from_file)) (<= (file_idx ?from_file) 7) (<= 3 (file_idx ?to_file)) (<= (file_idx ?to_file) 8) (= ?from_file ?to_file) (= (+ 1 (rank_idx ?from_rank)) (rank_idx ?to_rank)))
  :effect (and (not (at_ ?piece ?from_file ?from_rank)) (not (occupied ?from_file ?from_

In [7]:
print(writer.get_problem())

(define (problem development_of_pieces_in_chess-problem)
 (:domain development_of_pieces_in_chess-domain)
 (:objects
   a b c d e f g h - file
   s t u v w x y z - rank
   pa pb pc pd pe pf pg ph r1 r2 n1 n2 b1 b2 - piece
 )
 (:init (= (file_idx a) 1) (= (file_idx b) 2) (= (file_idx c) 3) (= (file_idx d) 4) (= (file_idx e) 5) (= (file_idx f) 6) (= (file_idx g) 7) (= (file_idx h) 8) (= (rank_idx s) 1) (= (rank_idx t) 2) (= (rank_idx u) 3) (= (rank_idx v) 4) (= (rank_idx w) 5) (= (rank_idx x) 6) (= (rank_idx y) 7) (= (rank_idx z) 8) (occupied a s) (occupied a t) (occupied b s) (occupied b t) (occupied c s) (occupied c t) (occupied d t) (occupied e t) (occupied f s) (occupied f t) (occupied g s) (occupied g t) (occupied h s) (occupied h t) (free a t) (free a u) (free a v) (free a w) (free a x) (free a y) (free a z) (free b t) (free b u) (free b v) (free b w) (free b x) (free b y) (free b z) (free c t) (free c u) (free c v) (free c w) (free c x) (free c y) (free c z) (free d s) (free d t) 

In [8]:
def run_planner_tamer(problem: Problem) -> None:
    with OneshotPlanner(name='tamer',
                        problem_kind=problem.kind) as planner:
        result = planner.solve(problem)
        print(f"Result: {result}")
        if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
            print("Plan found:")
            for action in result.plan.actions:
                print(f" - {action}")
        else:
            print(f"No plan found. Status: {result.status}")

#run_planner_tamer(problem)

In [9]:
def run_planner_pyperplan(problem: Problem) -> None:
    with OneshotPlanner(name='pyperplan',
                        problem_kind=problem.kind) as planner:
        result = planner.solve(problem)
        print(f"Result: {result}")
        if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
            print("Plan found:")
            for action in result.plan.actions:
                print(f" - {action}")
        else:
            print(f"No plan found. Status: {result.status}")

#run_planner_pyperplan(problem)

In [10]:
def run_planner_enhsp(problem: Problem) -> None:
    with OneshotPlanner(name='enhsp',
                        problem_kind=problem.kind) as planner:
        result = planner.solve(problem)
        print(f"Result: {result}")
        if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
            print("Plan found:")
            for action in result.plan.actions:
                print(f" - {action}")
        else:
            print(f"No plan found. Status: {result.status}")

#run_planner_enhsp(problem)

In [11]:
def run_planner_aries(problem: Problem) -> None:
    with OneshotPlanner(name='aries',
                        problem_kind=problem.kind) as planner:
        result = planner.solve(problem)
        print(f"Result: {result}")
        if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
            print("Plan found:")
            for action in result.plan.actions:
                print(f" - {action}")
        else:
            print(f"No plan found. Status: {result.status}")

#run_planner_aries(problem)

In [12]:
# fmap is a multi-agent planner and would require modifications to our problem
def run_planner_fmap(problem: Problem) -> None:
    with OneshotPlanner(name='fmap',
                        problem_kind=problem.kind) as planner:
        result = planner.solve(problem)
        print(f"Result: {result}")
        if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
            print("Plan found:")
            for action in result.plan.actions:
                print(f" - {action}")
        else:
            print(f"No plan found. Status: {result.status}")

#run_planner_fmap(problem)

In [13]:
def run_planner_fast_downward(problem: Problem) -> None:
    with OneshotPlanner(name='fast-downward',
                        problem_kind=problem.kind,
                        optimality_guarantee=PlanGenerationResultStatus.SOLVED_OPTIMALLY) as planner:
        result = planner.solve(problem)
        print(f"Result: {result}")
        if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
            print("Plan found:")
            for action in result.plan.actions:
                print(f" - {action}")
        else:
            print(f"No plan found. Status: {result.status}")

#run_planner_fast_downward(problem)

In [14]:
print(f"problem.kind={problem.kind}")

problem.kind=PROBLEM_CLASS: ['ACTION_BASED']
PROBLEM_TYPE: ['SIMPLE_NUMERIC_PLANNING']
NUMBERS: ['BOUNDED_TYPES']
CONDITIONS_KIND: ['EQUALITIES', 'DISJUNCTIVE_CONDITIONS', 'NEGATIVE_CONDITIONS', 'UNIVERSAL_CONDITIONS']
TYPING: ['FLAT_TYPING']
FLUENTS_TYPE: ['INT_FLUENTS']
INITIAL_STATE: ['UNDEFINED_INITIAL_SYMBOLIC']


In [15]:
writer = PDDLWriter(problem)
print(writer.get_domain())

(define (domain development_of_pieces_in_chess-domain)
 (:requirements :strips :typing :negative-preconditions :disjunctive-preconditions :equality :numeric-fluents :universal-preconditions)
 (:types file rank piece)
 (:predicates (occupied ?v_f_occ - file ?v_r_occ - rank) (free ?v_f_free - file ?v_r_free - rank) (at_ ?v_p_at - piece ?v_f_at - file ?v_r_at - rank) (moved ?v_p_moved - piece))
 (:functions (file_idx ?f - file) (rank_idx ?r - rank))
 (:action move_pawn_up_by_1
  :parameters ( ?piece - piece ?from_file - file ?from_rank - rank ?to_file - file ?to_rank - rank)
  :precondition (and (at_ ?piece ?from_file ?from_rank) (not (occupied ?to_file ?to_rank)) (free ?to_file ?to_rank) (< (rank_idx ?from_rank) 8) (<= 2 (file_idx ?from_file)) (<= (file_idx ?from_file) 7) (<= 3 (file_idx ?to_file)) (<= (file_idx ?to_file) 8) (= ?from_file ?to_file) (= (+ 1 (rank_idx ?from_rank)) (rank_idx ?to_rank)))
  :effect (and (not (at_ ?piece ?from_file ?from_rank)) (not (occupied ?from_file ?from_

In [16]:
print(writer.get_problem())

(define (problem development_of_pieces_in_chess-problem)
 (:domain development_of_pieces_in_chess-domain)
 (:objects
   a b c d e f g h - file
   s t u v w x y z - rank
   pa pb pc pd pe pf pg ph r1 r2 n1 n2 b1 b2 - piece
 )
 (:init (= (file_idx a) 1) (= (file_idx b) 2) (= (file_idx c) 3) (= (file_idx d) 4) (= (file_idx e) 5) (= (file_idx f) 6) (= (file_idx g) 7) (= (file_idx h) 8) (= (rank_idx s) 1) (= (rank_idx t) 2) (= (rank_idx u) 3) (= (rank_idx v) 4) (= (rank_idx w) 5) (= (rank_idx x) 6) (= (rank_idx y) 7) (= (rank_idx z) 8) (occupied a s) (occupied a t) (occupied b s) (occupied b t) (occupied c s) (occupied c t) (occupied d t) (occupied e t) (occupied f s) (occupied f t) (occupied g s) (occupied g t) (occupied h s) (occupied h t) (free a t) (free a u) (free a v) (free a w) (free a x) (free a y) (free a z) (free b t) (free b u) (free b v) (free b w) (free b x) (free b y) (free b z) (free c t) (free c u) (free c v) (free c w) (free c x) (free c y) (free c z) (free d s) (free d t) 